In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib.gridspec import GridSpec
pd.set_option('display.max_columns', 100)

import json
from wordcloud import WordCloud
from collections import Counter

# Feature Engineering

## Add Payment Method Frequency

Drop most frequent payment method

In [56]:
customer_rfm = pd.read_csv('Generated_Dataset/customer_rfm.csv')
customer_rfm.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,id,sigla,nome,nome_regiao,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,order_id,order_purchase_timestamp,order_purchase_year,order_purchase_month,order_purchase_month_name,order_purchase_year_month,order_purchase_date,order_purchase_day,order_purchase_dayofweek,order_purchase_dayofweek_name,order_purchase_hour,order_purchase_time_day,payment_type,payment_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,35,SP,São Paulo,Sudeste,14409,-20.498489,-47.396929,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,2017,5,May,201705,20170516,16,1,Tue,15,Afternoon,credit_card,146.87
1,5dca924cc99eea2dc5ba40d11ec5dd0f,2761fee7f378f0a8d7682d8a3fa07ab1,14409,franca,SP,35,SP,São Paulo,Sudeste,14409,-20.498489,-47.396929,95261f608a64bbbe760a212b4d48a4ee,2018-06-15 20:07:13,2018,6,Jun,201806,20180615,15,4,Fri,20,Night,credit_card,94.77
2,661897d4968f1b59bfff74c7eb2eb4fc,d06a495406b79cb8203ea21cc0942f8c,14409,franca,SP,35,SP,São Paulo,Sudeste,14409,-20.498489,-47.396929,9444fa2ab50a3f5af63b48be297eda24,2017-09-09 15:40:00,2017,9,Sep,201709,20170909,9,5,Sat,15,Afternoon,credit_card,62.50
3,702b62324327ccba20f1be3465426437,8b3d988f330c1d1c3332ccd440c147b7,14409,franca,SP,35,SP,São Paulo,Sudeste,14409,-20.498489,-47.396929,dceb8e88274c6f42a88a76ed979eb817,2018-03-26 12:04:55,2018,3,Mar,201803,20180326,26,0,Mon,12,Morning,credit_card,32.69
4,bdf997bae7ca819b0415f5174d6b4302,866755e25db620f8d7e81b351a15bb2f,14409,franca,SP,35,SP,São Paulo,Sudeste,14409,-20.498489,-47.396929,6ee1cea1b2edcc713f83ebfbccbc57f9,2018-07-29 20:39:20,2018,7,Jul,201807,20180729,29,6,Sun,20,Night,credit_card,77.01


In [57]:
# Calculate the percentage of each payment_type for each customer_unique_id
payment_type_percentage = customer_rfm.groupby(['customer_unique_id', 'payment_type']).size() / customer_rfm.groupby('customer_unique_id').size()
payment_type_percentage = payment_type_percentage.reset_index(name='payment_type_percentage')

# Pivot the payment_type_percentage dataframe to create features
payment_type_features = payment_type_percentage.pivot(index='customer_unique_id', columns='payment_type', values='payment_type_percentage').fillna(0)

In [58]:
customer_pay = customer_rfm.groupby('customer_unique_id').agg({
    'payment_type': lambda x: x.value_counts().index[0]}).reset_index().rename(columns={'payment_type': 'most_frequent_payment_types'})

customer_pay['n_payment_types'] = customer_rfm.groupby('customer_unique_id').payment_type.nunique().reset_index()['payment_type']

In [59]:
customer_pay['average_payment_value'] = customer_rfm.groupby('customer_unique_id').agg({'payment_value': 'mean'}).reset_index()['payment_value']
customer_pay['average_order_value'] = customer_rfm.groupby('customer_unique_id').agg({'payment_value': 'sum'}).reset_index()['payment_value']/customer_rfm.groupby('customer_unique_id').agg({'order_id': 'nunique'}).reset_index()['order_id']

In [60]:
customer_pay = customer_pay.merge(payment_type_features, on='customer_unique_id', how='inner')

In [61]:
customer_pay.drop(columns=['most_frequent_payment_types'], inplace=True)

In [73]:
customer_pay.head()

,customer_unique_id,n_payment_types,average_payment_value,average_order_value,boleto,credit_card,debit_card,not_defined,voucher
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90,141.90,0.0,1.0,0.0,0.0,0.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,27.19,0.0,1.0,0.0,0.0,0.0
2,0000f46a3911fa3c0805444483337064,1,86.22,86.22,0.0,1.0,0.0,0.0,0.0
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62,43.62,0.0,1.0,0.0,0.0,0.0
4,0004aac84e0df4da2b147fca70cf8255,1,196.89,196.89,0.0,1.0,0.0,0.0,0.0


In [62]:
customer_pay.to_csv('Generated_Dataset/cutomer_pay.csv', index=False)

## Add Product Purchasing Frequency

In [51]:
orders = pd.read_csv('Dataset/olist_orders_dataset.csv')
customers = pd.read_csv('Dataset/olist_customers_dataset.csv')
product_category = pd.read_csv('Generated_Dataset/product_category.csv')

order_customer = pd.merge(orders, customers, on='customer_id')
customer_product = pd.merge(order_customer, product_category, on='order_id', how='left')

In [52]:
customer_product.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_id,product_category_name,order_item_id,seller_id,shipping_limit_date,price,freight_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,review_creation_year,review_creation_month,review_creation_month_name,review_creation_year_month,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,87285b34884572647811a353c7ac498a,utilidades_domesticas,1.0,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",20171011.0,2017-10-12 03:43:48,2017.0,10.0,Oct,201710.0,housewares
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,595fac2a385ac33a80bd5114aec74eb8,perfumaria,1.0,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,20180808.0,2018-08-08 18:37:50,2018.0,8.0,Aug,201808.0,perfumery
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,aa4383b373c6aca5d8797843e5594415,automotivo,1.0,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,NaN,20180818.0,2018-08-22 19:07:58,2018.0,8.0,Aug,201808.0,auto
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,d0b61bfb1de832b15ba9d266ca96e5b0,pet_shop,1.0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,359d03e676b3c069f62cadba8dd3f6e8,5.0,NaN,O produto foi exatamente o que eu esperava e e...,20171203.0,2017-12-05 19:21:58,2017.0,12.0,Dec,201712.0,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,65266b2da20d04dbe00c5c2d3bb7859e,papelaria,1.0,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,e50934924e227544ba8246aeb3770dd4,5.0,NaN,NaN,20180217.0,2018-02-18 13:02:51,2018.0,2.0,Feb,201802.0,stationery


In [54]:
HomeLiving = ['home_appliances','home_appliances_2','small_appliances','bed_bath_table', 'furniture_decor', 'housewares', 'garden_tools', 'office_furniture', 'home_construction', 'furniture_living_room', 'home_confort', 'kitchen_dining_laundry_garden_furniture', 'furniture_bedroom', 'small_appliances_home_oven_and_coffee', 'home_comfort_2', 'furniture_mattress_and_upholstery', 'la_cuisine','food','drinks','food_drink','security_and_services']
BeautyFashion = ['health_beauty', 'perfumery', 'baby', 'diapers_and_hygiene', 'fashion_bags_accessories',  'pet_shop', 'luggage_accessories', 'fashion_shoes', 'fashion_underwear_beach', 'fashion_male_clothing', 'fashio_female_clothing', 'fashion_sport', 'fashion_childrens_clothes', 'arts_and_craftmanship']
Electronics = ['computers_accessories', 'watches_gifts', 'telephony', 'fixed_telephony','electronics', 'consoles_games', 'audio', 'computers', 'tablets_printing_image', 'cine_photo','air_conditioning']
LeisureEntertainment = ['sports_leisure', 'toys', 'cool_stuff','pet_shop', 'musical_instruments', 'art', 'flowers', 'party_supplies', 'arts_and_craftmanship', 'cds_dvds_musicals', 'music', 'christmas_supplies', 'dvds_blu_ray', 'books_imported', 'books_general_interest', 'books_technical', 'cds_dvds_musicals', 'music']
Tools = ['stationery','auto', 'construction_tools_construction', 'industry_commerce_and_business', 'construction_tools_lights', 'construction_tools_safety', 'signaling_and_security', 'costruction_tools_garden', 'agro_industry_and_commerce', 'costruction_tools_tools','market_place']

customer_product['product_category_group'] = customer_product['product_category_name_english'].apply(lambda x: 'Home & Living' if x in HomeLiving else ('Beauty & Fashion' if x in BeautyFashion else ('Electronics' if x in Electronics else ('Leisure & Entertainment' if x in LeisureEntertainment else ('Tools' if x in Tools else 'Missing')))))
## 'Missing' denotes missing values


In [67]:
# Calculate the percentage of each payment_type for each customer_unique_id
product_category_percentage = customer_product.groupby(['customer_unique_id', 'product_category_group']).size() / customer_product.groupby('customer_unique_id').size()
product_category_percentage = product_category_percentage.reset_index(name='product_category_percentage')

# Pivot the payment_type_percentage dataframe to create features
product_category_features = product_category_percentage.pivot(index='customer_unique_id', columns='product_category_group', values='product_category_percentage').fillna(0)

In [68]:
product_category_features

product_category_group,Beauty & Fashion,Electronics,Home & Living,Leisure & Entertainment,Missing,Tools
customer_unique_id,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,0.0,0.0,1.0,0.0,0.0,0.0
0000b849f77a49e4a4ce2b2a4ca5be3f,1.0,0.0,0.0,0.0,0.0,0.0
0000f46a3911fa3c0805444483337064,0.0,0.0,0.0,0.0,0.0,1.0
0000f6ccb0745a6a4b88665a16c9f078,0.0,1.0,0.0,0.0,0.0,0.0
0004aac84e0df4da2b147fca70cf8255,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,1.0,0.0,0.0,0.0,0.0,0.0
fffea47cd6d3cc0a88bd621562a9d061,1.0,0.0,0.0,0.0,0.0,0.0
ffff371b4d645b6ecea244b27531430a,0.0,0.0,0.0,0.0,0.0,1.0


In [74]:
product_category_features = product_category_features.reset_index()


In [76]:
product_category_features

product_category_group,customer_unique_id,Beauty & Fashion,Electronics,Home & Living,Leisure & Entertainment,Missing,Tools
0,0000366f3b9a7992bf8c76cfdf3221e2,0.0,0.0,1.0,0.0,0.0,0.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1.0,0.0,0.0,0.0,0.0,0.0
2,0000f46a3911fa3c0805444483337064,0.0,0.0,0.0,0.0,0.0,1.0
3,0000f6ccb0745a6a4b88665a16c9f078,0.0,1.0,0.0,0.0,0.0,0.0
4,0004aac84e0df4da2b147fca70cf8255,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
96091,fffcf5a5ff07b0908bd4e2dbc735a684,1.0,0.0,0.0,0.0,0.0,0.0
96092,fffea47cd6d3cc0a88bd621562a9d061,1.0,0.0,0.0,0.0,0.0,0.0
96093,ffff371b4d645b6ecea244b27531430a,0.0,0.0,0.0,0.0,0.0,1.0
96094,ffff5962728ec6157033ef9805bacc48,0.0,1.0,0.0,0.0,0.0,0.0


## Merge Datasets

In [70]:
customer_review_agg = pd.read_csv('Generated_Dataset/customer_review_agg.csv')
rfm = pd.read_csv('Generated_Dataset/rfm.csv')

In [88]:
customer_agg = pd.merge(customer_review_agg, customer_pay, on='customer_unique_id')
customer_agg = pd.merge(customer_agg, product_category_features, on='customer_unique_id')
customer_agg = pd.merge(customer_agg, rfm, on='customer_unique_id')
customer_agg.head()

,customer_unique_id,avg_review_score,order_counts,review_score_counts,review_comment_counts,rating_rate,comment_rate,n_payment_types,average_payment_value,average_order_value,boleto,credit_card,debit_card,not_defined,voucher,Beauty & Fashion,Electronics,Home & Living,Leisure & Entertainment,Missing,Tools,order_purchase_timestamp,Recency,Frequency,Monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,5.0,1,1,1,1.0,1.0,1,141.90,141.90,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018-05-10 10:56:27,160,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,4.0,1,1,0,1.0,0.0,1,27.19,27.19,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2018-05-07 11:11:27,163,1,27.19
2,0000f46a3911fa3c0805444483337064,3.0,1,1,0,1.0,0.0,1,86.22,86.22,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2017-03-10 21:05:03,585,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,4.0,1,1,1,1.0,1.0,1,43.62,43.62,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-10-12 20:29:41,369,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,5.0,1,1,0,1.0,0.0,1,196.89,196.89,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-11-14 19:45:42,336,1,196.89


In [79]:
customer_agg.to_csv('Generated_Dataset/customer_agg_4cluster.csv', index=False)

# Data Inspection

## Check Missing

In [80]:
customer_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95826 entries, 0 to 95825
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_unique_id        95826 non-null  object 
 1   avg_review_score          95112 non-null  float64
 2   order_counts              95826 non-null  int64  
 3   review_score_counts       95826 non-null  int64  
 4   review_comment_counts     95826 non-null  int64  
 5   rating_rate               95826 non-null  float64
 6   comment_rate              95826 non-null  float64
 7   n_payment_types           95826 non-null  int64  
 8   average_payment_value     95826 non-null  float64
 9   average_order_value       95826 non-null  float64
 10  boleto                    95826 non-null  float64
 11  credit_card               95826 non-null  float64
 12  debit_card                95826 non-null  float64
 13  not_defined               95826 non-null  float64
 14  vouche

In [81]:
customer_agg.isna().sum()

customer_unique_id            0
avg_review_score            714
order_counts                  0
review_score_counts           0
review_comment_counts         0
rating_rate                   0
comment_rate                  0
n_payment_types               0
average_payment_value         0
average_order_value           0
boleto                        0
credit_card                   0
debit_card                    0
not_defined                   0
voucher                       0
Beauty & Fashion              0
Electronics                   0
Home & Living                 0
Leisure & Entertainment       0
Missing                       0
Tools                         0
order_purchase_timestamp      0
Recency                       0
Frequency                     0
Monetary                      0
dtype: int64

In [90]:
# filter rows with not defined payment methods or missing product categories

customer_agg = customer_agg[(customer_agg['not_defined'] == 0) & (customer_agg['Missing'] == 0)].reset_index(drop=True)
customer_agg.head()

,customer_unique_id,avg_review_score,order_counts,review_score_counts,review_comment_counts,rating_rate,comment_rate,n_payment_types,average_payment_value,average_order_value,boleto,credit_card,debit_card,not_defined,voucher,Beauty & Fashion,Electronics,Home & Living,Leisure & Entertainment,Missing,Tools,order_purchase_timestamp,Recency,Frequency,Monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,5.0,1,1,1,1.0,1.0,1,141.90,141.90,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018-05-10 10:56:27,160,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,4.0,1,1,0,1.0,0.0,1,27.19,27.19,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2018-05-07 11:11:27,163,1,27.19
2,0000f46a3911fa3c0805444483337064,3.0,1,1,0,1.0,0.0,1,86.22,86.22,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2017-03-10 21:05:03,585,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,4.0,1,1,1,1.0,1.0,1,43.62,43.62,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-10-12 20:29:41,369,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,5.0,1,1,0,1.0,0.0,1,196.89,196.89,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-11-14 19:45:42,336,1,196.89


In [92]:
# drop rows with NA values

customer_agg.dropna(inplace=True)


In [94]:
customer_agg.shape

(92922, 25)

## Drop Unuseful Columns

In [95]:
customer_agg.head()

,customer_unique_id,avg_review_score,order_counts,review_score_counts,review_comment_counts,rating_rate,comment_rate,n_payment_types,average_payment_value,average_order_value,boleto,credit_card,debit_card,not_defined,voucher,Beauty & Fashion,Electronics,Home & Living,Leisure & Entertainment,Missing,Tools,order_purchase_timestamp,Recency,Frequency,Monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,5.0,1,1,1,1.0,1.0,1,141.90,141.90,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018-05-10 10:56:27,160,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,4.0,1,1,0,1.0,0.0,1,27.19,27.19,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2018-05-07 11:11:27,163,1,27.19
2,0000f46a3911fa3c0805444483337064,3.0,1,1,0,1.0,0.0,1,86.22,86.22,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2017-03-10 21:05:03,585,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,4.0,1,1,1,1.0,1.0,1,43.62,43.62,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-10-12 20:29:41,369,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,5.0,1,1,0,1.0,0.0,1,196.89,196.89,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-11-14 19:45:42,336,1,196.89


In [96]:
customer_agg.drop(columns=['average_order_value', 'not_defined', 'Missing'], inplace=True)